In [11]:
# Rescaling each pixel to be larger than existing

import rasterio
from rasterio.enums import Resampling
import pandas as pd
import numpy as np
import os
from pyproj import Transformer

data_directory = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\test_data"
scale_factor = 100 / 30  # Change 100 to your desired output resolution

# Loop through each file in the directory
for filename in os.listdir(data_directory):
    if filename.startswith("L8_UTC_"):  # Adjust to match your filename structure
        file_path = os.path.join(data_directory, filename)

        # Open the GeoTIFF file
        with rasterio.open(file_path) as src:
            # Resample the data
            data = src.read(
                out_shape=(
                    src.count,
                    int(src.height / scale_factor),
                    int(src.width / scale_factor)
                ),
                resampling=Resampling.cubic  # Or Resampling.bilinear
            )

            # Adjust the transformation for resampled resolution
            transform = src.transform * src.transform.scale(
                (src.width / data.shape[-1]),
                (src.height / data.shape[-2])
            )

            # Create transformer for coordinate conversion
            transformer = Transformer.from_crs(src.crs, 'EPSG:4326', always_xy=True)
            cols, rows = np.meshgrid(np.arange(data.shape[-1]), np.arange(data.shape[-2]))
            xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')
            lon, lat = transformer.transform(np.array(xs).flatten(), np.array(ys).flatten())

            # Flatten and create DataFrame
            reshaped_array = data.transpose(1, 2, 0).reshape(-1, data.shape[0])
            df = pd.DataFrame(reshaped_array, columns=src.descriptions)
            df['Longitude'] = lon
            df['Latitude'] = lat
            df['ST_B10_Celsius'] = df['ST_B10'] * 0.00341802 + 149 - 273.15

            # Adjusting band values as before
            bands_to_scale = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
            for band in bands_to_scale:
                df[f"{band}_Scaled"] = df[band] * 2.75e-05 - 0.2

            additional_scales = {
                'ST_ATRAN': 0.0001, 'ST_CDIST': 0.01, 'ST_DRAD': 0.001, 
                'ST_EMIS': 0.0001, 'ST_EMSD': 0.0001, 'ST_QA': 0.01, 
                'ST_TRAD': 0.001, 'ST_URAD': 0.001
            }
            for band, scale in additional_scales.items():
                df[f"{band}_Scaled"] = df[band] * scale

            # # Save the bands data to CSV
            # bands_csv_path = os.path.join(data_directory, f'Landsat8_Bands_{filename[:-4]}.csv')
            # bands_df.to_csv(bands_csv_path, index=False)
            # print(f"Saved band data to {bands_csv_path}") 

            # # Save metadata to CSV
            # metadata_df = pd.DataFrame([src.meta])
            # metadata_csv_path = os.path.join(data_directory, f'Landsat8_Metadata_{filename[:-4]}.csv')
            # metadata_df.to_csv(metadata_csv_path, index=False)
            # print(f"Saved metadata to {metadata_csv_path}")

print(df[['Latitude', 'Longitude']].head())
print(len(df))

   Latitude   Longitude
0  1.469782  103.590066
1  1.469783  103.590966
2  1.469783  103.591866
3  1.469784  103.592766
4  1.469784  103.593665
200784


In [12]:
import geopandas as gpd

# First, clean your data by dropping or filling NaN values
df = df.dropna(subset=['ST_B10_Celsius'])  # Removes rows where 'ST_B10_Celsius' is NaN
singapore_geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\singapore-boundary.geojson"

# Create a map centered around the mean coordinates of the filtered data
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]

singapore_boundary = gpd.read_file(singapore_geojson_path)

# Convert your DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
gdf.crs = singapore_boundary.crs  # Ensure the CRS matches

# Perform spatial join to filter points within Singapore
gdf = gpd.sjoin(gdf, singapore_boundary, how="inner", op='within')

c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [13]:
# Heatmap of rescaled graph where each pixel represents a bigger area

import geopandas as gpd
import hvplot.pandas
import holoviews as hv
from bokeh.palettes import Turbo256  # Import a predefined Bokeh palette

# Assuming 'gdf' is your preloaded GeoDataFrame
gdf = gdf.to_crs(epsg=3857)  # Convert to Web Mercator for better mapping support

# Define a function to select a subset of the color palette
def select_colors(palette, n):
    return [palette[int(i)] for i in np.linspace(0, len(palette)-1, n)]

# Create a custom color scale using a continuous palette
custom_palette = select_colors(Turbo256, 256)  # More colors for smoother transitions

# Create the heatmap
heatmap = gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius',
                            cmap=custom_palette, size=5,  # Smaller size for finer detail
                            tiles='OSM', frame_width=700, frame_height=500,
                            colorbar=True, clim=(20, 40))

#file_path = "C:/LocalOneDrive/Documents/Desktop/MTI/UHI-Project/MSE-ES-UHI/MSE-ES-UHI/2_landsat/Heatmaps"

# Set up Panel to display the plot
heatmap_panel = hv.save(heatmap, 'hvPlot_Land_Surface_Temperature_Map_coarse_gradient.html', backend='bokeh')

# Display the plot in the notebook
heatmap_panel